In [62]:
import pandas as pd
import os
import glob
from functools import reduce
from datetime import date, datetime
import plotly.express as px
import numpy as np

In [53]:
stocks = []
companies = []
import glob
for file in glob.glob("./Companies/*"):
    company_name = os.path.basename(file).split(".")[0]
    companies.append(company_name)
    stock = pd.read_csv(file)
    stock = stock[["Date", "Close"]]
    stock.rename(columns={"Close": company_name}, inplace=True)
    stock.set_index(["Date"],inplace=True)
    stocks.append(stock)
    
merged_df = reduce(lambda left,right: pd.merge(left,right, how='outer', left_index=True, right_index=True), stocks)
merged_df = merged_df.drop(labels="2021-11-19", axis=0) # Incomplete data for yesterday

In [54]:
# Verifying nans are only dates before IPO.
for company in companies:
    if "2019-01-02" !=  merged_df[company].first_valid_index():
        nanIndex = merged_df[company][merged_df[company].isnull()].index.tolist()
        print(len(nanIndex))
        start = datetime.strptime(merged_df[company].first_valid_index(), '%Y-%m-%d').date()
        print(start)
        delta = (start - date(2019, 1, 2))
        print(delta.days*5/7)
        print(company, merged_df[company].first_valid_index(), merged_df[company].last_valid_index())

60
2019-03-29
61.42857142857143
LYFT 2019-03-29 2021-11-18
490
2020-12-10
505.7142857142857
ABNB 2020-12-10 2021-11-18
565
2021-03-31
585.0
COUR 2021-03-31 2021-11-18
89
2019-05-10
91.42857142857143
UBER 2019-05-10 2021-11-18
74
2019-04-18
75.71428571428571
ZM 2019-04-18 2021-11-18


In [55]:
merged_df

,UAL,V,LYFT,COST,ABNB,AMZN,COUR,UBER,AXP,MAR,TWTR,NFLX,AAL,ZM,DAL,MA,HLT,WMT
Date,,,,,,,,,,,,,,,,,,
2019-01-02,84.180000,132.919998,NaN,204.759995,NaN,1539.130005,NaN,NaN,95.680000,107.459999,28.809999,267.660004,32.480000,NaN,50.090000,189.740005,70.940002,93.339996
2019-01-03,80.000000,128.130005,NaN,200.419998,NaN,1500.280029,NaN,NaN,93.430000,101.739998,27.990000,271.200012,30.059999,NaN,45.610001,181.179993,67.300003,92.860001
2019-01-04,82.680000,133.649994,NaN,206.240005,NaN,1575.390015,NaN,NaN,97.639999,107.809998,29.950001,297.570007,32.040001,NaN,47.790001,189.759995,69.940002,93.440002
2019-01-07,83.230003,136.059998,NaN,207.000000,NaN,1629.510010,NaN,NaN,98.169998,108.010002,31.340000,315.339996,32.950001,NaN,47.849998,191.220001,70.739998,94.540001
2019-01-08,82.379997,136.800003,NaN,208.550003,NaN,1656.579956,NaN,NaN,98.650002,109.760002,31.799999,320.269989,32.419998,NaN,47.470001,192.279999,71.860001,95.199997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-12,49.830002,212.089996,52.959999,517.169983,206.539993,3525.149902,35.230000,45.119999,181.889999,156.479996,52.250000,682.609985,20.290001,254.970001,42.220001,363.500000,143.470001,147.759995
2021-11-15,50.480000,212.300003,51.660000,519.890015,207.210007,3545.679932,35.740002,43.610001,183.130005,155.679993,52.950001,679.330017,20.320000,263.709991,42.380001,361.359985,142.910004,146.910004
2021-11-16,49.200001,215.179993,51.279999,526.719971,207.039993,3540.699951,36.540001,45.759998,180.919998,153.990005,52.110001,687.400024,19.840000,264.489990,41.040001,369.559998,141.740005,143.169998


In [56]:
merged_df.to_csv("./Data/DailyStockData.csv")


In [63]:
mean_norm_df = (merged_df-merged_df.mean())/merged_df.std()
inc_df = merged_df.diff(axis=0)/merged_df
min_df = merged_df-merged_df.min()
max_df = merged_df-merged_df.max()
log_df = np.log(merged_df)

In [58]:
dict_stock_to_company = {}
dict_stock_to_company['UAL'] = 'United Airlines'
dict_stock_to_company['V'] = 'Visa'
dict_stock_to_company['LYFT'] = 'Lyft'
dict_stock_to_company['COST'] = 'Costco'
dict_stock_to_company['ABNB'] = 'Airbnb'
dict_stock_to_company['AMZN'] = 'Amazon'
dict_stock_to_company['COUR'] = 'Coursera'
dict_stock_to_company['UBER'] = 'Uber'
dict_stock_to_company['AXP'] = 'American Express'
dict_stock_to_company['MAR'] = 'Marriot'
dict_stock_to_company['TWTR'] = 'Twitter'
dict_stock_to_company['NFLX'] = 'Netflix'
dict_stock_to_company['AAL'] = 'American Airlines'
dict_stock_to_company['ZM'] = 'Zoom'
dict_stock_to_company['DAL'] = 'Delta Airlines'
dict_stock_to_company['MA'] = 'Mastercard'
dict_stock_to_company['HLT'] = 'Hilton'
dict_stock_to_company['WMT'] = 'Walmart'
dict_industry_to_stock = {}
dict_industry_to_stock['Transport'] = ['UBER', 'LYFT']
dict_industry_to_stock['Hospitality'] = ['ABNB', 'MAR', 'HLT']
dict_industry_to_stock['Consumer Spending'] = ['V', 'AXP', 'MA']
dict_industry_to_stock['Commerce'] = ['AMZN', 'WMT', 'COST']
dict_industry_to_stock['Online Services'] = ['COUR', 'ZM', 'TWTR', 'NFLX']
dict_industry_to_stock['Airlines'] = ['UAL', 'DAL', 'AAL']


In [64]:
df_temp = pd.melt(log_df.reset_index(), id_vars=['Date'], value_vars=dict_stock_to_company.keys()).rename(columns={"variable": "Company", "value": "Stock Price"}).replace({"Company": dict_stock_to_company})
fig = px.line(df_temp, x="Date", y="Stock Price", color='Company', title="Stock price comparison for considered companies")
fig.show()